In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
DATA_DIR = '../input/ear-dataset/EarVN1.0 dataset/Images'
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

In [3]:
train_val_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.15,
  label_mode='int',
  subset="training",
  seed=999,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 28412 files belonging to 164 classes.
Using 24151 files for training.


In [4]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.15,
  label_mode='int',
  subset="validation",
  seed=999,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 28412 files belonging to 164 classes.
Using 4261 files for validation.


In [5]:
train_val_batches = tf.data.experimental.cardinality(train_val_ds)
train_val_batches

<tf.Tensor: shape=(), dtype=int64, numpy=755>

In [6]:
TOTAL_TRAIN_VAL = len(train_val_ds)
print(TOTAL_TRAIN_VAL)
train_ds = train_val_ds.take(int(8 * train_val_batches / 10))
val_ds = train_val_ds.skip(int(8 * train_val_batches / 10))
print(len(train_ds))
print(len(val_ds))

755
604
151


In [7]:
from keras.models import load_model

model_path = '/kaggle/input/best-efficientnetv2s-model-epoch96-acc0-9158/best_efficientnetv2s_model_epoch96_acc0.9158.h5'

model = load_model(model_path)

In [8]:
predictions = []
y_true = []
for img, labels in test_ds:
    predictions.extend(model.predict_on_batch(img))
    y_true.extend(labels.numpy())
y_true = np.array(y_true)

In [9]:
y_pred = np.argmax(predictions, axis=1)

In [10]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.86      0.89        50
           1       0.88      0.97      0.93        39
           2       0.91      0.83      0.87        24
           3       0.97      0.97      0.97        31
           4       0.94      0.91      0.93        34
           5       1.00      0.94      0.97        35
           6       1.00      0.96      0.98        23
           7       1.00      0.81      0.89        26
           8       0.92      0.94      0.93        35
           9       1.00      1.00      1.00        28
          10       1.00      0.91      0.95        23
          11       0.88      1.00      0.94        29
          12       0.96      0.92      0.94        24
          13       0.94      0.94      0.94        17
          14       0.97      0.91      0.94        34
          15       0.96      0.96      0.96        28
          16       0.79      0.96      0.86        23
          17       0.80    